# New approach, find lines with correct format (with thanks to David Kane at https://github.com/drkane)

Find correct sheets by finding those titled "Balance Sheet"

Exclude a bunch of false positives by filtering out "Statement of" and "Note to the financial statements"

Detect aligned elements by whitespace

within concatenated elements, detect values by regex

if needed, complete multi-line label by finding correct aggregated sentence in blocks / pars

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import pytesseract                            # API for letting python interface with Google's tesseract OCR software
import re
import importlib

import xbrl_image_parser as xip



## 1. Get the filenames of the example data for my convenience

In [2]:
# Get a list of all of the pdf files in the directory "CH_records"
files = [filename.split(".")[0] for filename in os.listdir("./working/ocr_output_compiled") if ".csv" in filename]

In [3]:
for each in range(len(files)):
    print(each, files[each])

0 00002404
1 868273
2 1983517
3 2765595
4 03293902
5 02959325
6 00542515
7 01539777
8 02714555
9 00030177
10 04802747
11 02266230
12 00983951
13 01002610
14 01804186
15 5508774
16 02430955
17 00053475
18 02245999
19 00553535
20 3387163
21 01337451
22 3459907
23 00178090
24 00468115
25 01369166
26 00782931
27 09457025
28 983951
29 01370175
30 06005142
31 04860660
32 2303730
33 02582534
34 00477955
35 04558828
36 06034603
37 3824626


## 2. Read in a csv file of data extracted from a PDF

In [5]:
# Rediscovering what works...
importlib.reload(xip)
index=0

# So far can create all those extra geometric features, can convert to numeric
test = pd.read_csv("./working/ocr_output_compiled/"+files[index]+".csv")

In [9]:
# Create numeric vars where possible
test['numerical'] = xip.convert_to_numeric(test['text'])

# Do some geometry (eg; calculate bottom-bounding coordinates explicitly)
test = xip.make_measurements(test)

# Create a table of aggregated, multiline sentences
agg_text = xip.aggregate_sentences_over_lines(test)

In [12]:
# Look at the first page of the balance sheet.  Some things to note; that first element is a bounding box
# for the entire page
test[test['csv_num']==14][['top', 'bottom', 'text', 'level', 'conf']]

,top,bottom,text,level,conf
4614,0,3509,NaN,1,-1
4615,303,342,NaN,2,-1
4616,303,342,NaN,3,-1
4617,303,342,NaN,4,-1
4618,303,340,Southampton,5,93
4619,304,333,Isle,5,94
4620,304,333,of,5,96
4621,304,341,Wight,5,95
4622,304,333,and,5,96
4623,304,333,South,5,96


## 3. Identify balance sheet pages

## 4.  Find lines on the balance sheet

In [19]:
# first; the line detection algorithm
# Lifted this almost directly from David Kane's work
def detect_lines(page_df, x_tolerance=0):
    """
    Detect lines in the csv of a page, returned by Tesseract
    """
    words_df = page_df[page_df['word_num'] > 0]
    page_stats = page_df.iloc[0, :]
    
    row_ranges = []
    this_range = []
    
    # Iterate through every vertical pixel position, top (0) to bottom (height)
    for i in range(page_stats['height']):
        result = (( words_df['bottom'] >= i ) & ( words_df['top'] <= i )).sum() > 0
        
        # Append vertical pixels aligned with words to this_range
        if result:
            this_range.append(i)
        
        # If we've passed out of an "occupied" range, append the resulting range to a list to store
        else:
            if this_range:
                row_ranges.append(this_range)
            this_range = []
        
    # Create bounding boxes for convenience
    return[{"left":0, "right":page_stats['width'], "top":min(r), "bottom":max(r)} for r in row_ranges]

In [172]:
def extract_lines(page_df, lines):
    
    words_df = page_df[page_df['word_num'] > 0]
    
    results = pd.DataFrame()
    for line in lines:
        
        # Retrieve all text in line
        inline = (words_df['bottom'] <= line['bottom']) & (words_df['top'] >= line['top'])
        line_text = " ".join( words_df[inline]['text'] ).replace("(", "").\
                                                         replace(")", "").\
                                                         replace(",", "")
        
        # Get the text at the start of the line (presumed label)
        label = " ".join([i for i in line_text.split(" ") if not i.isnumeric()])
        
        # Get the aligned values
        values = [float(i) for i in line_text.split(" ") if i.isnumeric()]
        print(label, values)
        try:
            if label != "":
                results = results.append({"label":label, "CurrYr":values[-2], "LastYr":values[-1]}, ignore_index=True)
        except:
            pass
    
    return(results)

In [173]:
lines = extract_lines(page_df, detect_lines(page_df))

Southampton Isle of Wight and South of England Royal Mail Steam Packet Company Limited []
Registered number:00002404 []
Balance Sheet []
As at December — [31.0, 2015.0]
 [2015.0, 2014.0]
Note £000 £000 []
Fixed assets []
Intangible assets [12.0, 180.0, 137.0]
Tangible assets [13.0, 28271.0, 28681.0]
Investments [14.0]
 [28454.0, 28821.0]
Current assets []
Stocks [15.0, 446.0, 405.0]
Debtors: amounts falling due within one year [16.0, 23174.0, 11926.0]
Cash at bank and in hand [10208.0, 10565.0]
 [33828.0, 22896.0]
Creditors: amounts falling due within one year [17.0, 10936.0, 7527.0]
Net current assets [22892.0, 15369.0]
Total assets less current liabilities [51346.0, 44190.0]
"Creditors: amounts falling due after more []
than one year [18.0, 2620.0, 3071.0]
Provisions for liabilities []
Pensions [18.0, 18.0]
Pension liability [23.0, 2128.0, 2244.0]
Net assets [46580.0, 38857.0]
Page [11.0]


In [174]:
lines

,CurrYr,LastYr,label
0,31.0,2015.0,As at December —
1,180.0,137.0,Intangible assets
2,28271.0,28681.0,Tangible assets
3,446.0,405.0,Stocks
4,23174.0,11926.0,Debtors: amounts falling due within one year
5,10208.0,10565.0,Cash at bank and in hand
6,10936.0,7527.0,Creditors: amounts falling due within one year
7,22892.0,15369.0,Net current assets
8,51346.0,44190.0,Total assets less current liabilities
9,2620.0,3071.0,than one year


In [ ]:
# Next steps;
# make sure to add in missing label text from multi-line labels
# Detect if line has words between numbers (therefore not data)